In [ ]:
#Intensivão Python - Dia 1

print('Manipulando dados e automatizando processos no Python')

In [ ]:
#Importando a biblioteca pandas

import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Colab Notebooks/vendas/Vendas.xlsx')
df.head(3)

In [ ]:
display(df)

In [ ]:
#Calculando o faturamento por loja
#Colunas: Código Venda	Data	ID Loja	Produto	Quantidade	Valor Unitário	Valor Final

faturamento = df[['ID Loja', 'Valor Final']].groupby('ID Loja').sum()
faturamento

In [ ]:
#Ordenando por maior faturamento
faturamento = faturamento.sort_values(by='Valor Final', ascending=False)
faturamento

In [ ]:
#Calculando a quantidade vendida por loja
#Colunas: Código Venda	Data	ID Loja	Produto	Quantidade	Valor Unitário	Valor Final

#ordenei por loja com maior quantidade vendida

quantidade = df[['ID Loja','Quantidade']].groupby('ID Loja').sum()
quantidade = quantidade.sort_values(by='Quantidade', ascending=False)
display(quantidade)

In [ ]:
#Obtendo o ticket médio das lojas
ticket_medio = faturamento['Valor Final'] / quantidade['Quantidade']
ticket_medio

In [ ]:
#Obtendo o ticket médio das lojas
#usar o to_frame() para uma apresentação melhor da tabela
ticket_medio = (faturamento['Valor Final'] / quantidade['Quantidade']).to_frame()
ticket_medio

In [ ]:
#Obtendo o ticket médio das lojas
#Atribuindo nome para a coluna (Ticket Médio) e ordenar por maior ticket médio.
ticket_medio = (faturamento['Valor Final'] / quantidade['Quantidade']).to_frame()
ticket_medio = ticket_medio.rename(columns={0: 'Ticket Médio'})
ticket_medio

In [ ]:


#para enviar e-mail (automatizando processos)



In [ ]:
import smtplib
import email.message

In [ ]:
def enviar_email(resumo_loja, loja):

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f'''
  <p>Olá tudo bem? Segue abaixo as informações:</p>
  {resumo_loja.to_html()}
  <p>Qualquer dúvida estou à disposição. Att.</p>'''
    
  msg = email.message.Message()
  msg['Subject'] = f'Envio de relatórios - Loja: {loja}'
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
    # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = 'email@emailtestestes.com'
  msg['To'] = 'email1@emailtestestes.com'
  password = "#senha"
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')




# email para diretoria

tabela_diretoria = faturamento.join(quantidade).join(ticket_medio)
enviar_email(tabela_diretoria, 'Todas as Lojas')

In [ ]:
#email para todas as lojas

lojas = df['ID Loja'].unique()

for loja in lojas:
  tabela_loja = df.loc[df['ID Loja'] == loja, ['ID Loja', 'Quantidade', 'Valor Final']]
  resumo_loja = tabela_loja.groupby('ID Loja').sum()
  resumo_loja['Ticket Médio'] = resumo_loja['Valor Final'] / resumo_loja['Quantidade']
  enviar_email(resumo_loja, loja)